In [1]:
import json
import os

# Load configuration from a JSON file
with open('GradientConfig.json') as config_file:
    config = json.load(config_file)

os.environ['GRADIENT_ACCESS_TOKEN'] = config.get('GRADIENT_ACCESS_TOKEN')
os.environ['GRADIENT_WORKSPACE_ID'] = config.get('GRADIENT_WORKSPACE_ID')

In [2]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore

In [3]:
import cassandra
print (cassandra.__version__)

3.29.0


In [4]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-end-to-end-pdf-chat-using-llamaindex-llama-2.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("End to End PDF Chat using LlamaIndex & Llama 2-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.11-c7d4aa9d7ae5


In [5]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

In [7]:
embed_model = GradientEmbedding(
    gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

service_context = ServiceContext.from_defaults(
    chunk_size=256, llm=llm, embed_model=embed_model
)

set_global_service_context(service_context)
#%%
# Replace '/path/to/your/documents' with the actual path to your PDF documents on your local machine
documents_directory = './PDF Documents'

documents = SimpleDirectoryReader(documents_directory).load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 101 document(s).


In [8]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

In [9]:
response = query_engine.query("What is the total of all the cement costs in the file along with the name of the work?")

print(response)


The total of all the cement costs in the file along with the name of the work is:

Total cement cost = 1423049.68 + 3039523.18 + 48431.00 = 1855813.86

Name of the work:

* DSR 5.33.1.1: Upto plinth level concrete of M25 grade with minimum cement content of 330 kg /cum - 189.53 Cum x 330 kg/cum = 59,219.50
* DSR 5.33.2.1: Above plinth level upto floor V level concrete of M25 grade with minimum cement content of 330 kg /cum - 390.22 Cum x 330 kg/cum = 142,304.96
* 5.9 DSR: Centering and shuttering including strutting, propping etc. and removal of form for foundations, footings, bases of columns, etc. for mass concrete - 156.34 sqm x 330 kg/cum = 50,305.20

Therefore, the total of all the cement costs in the file along with the name of the work is 1855813.86.
